# 问题描述

使用OpenAI Gym中的CartPole，如下图所示：

![SegmentLocal](./CartPole.gif "segment")

本问题的状态包含四个元素：滑块的位置，滑块的速度，杆子的角度，杆子顶端的速度。动作包括滑块向左或向右移动。当杆子保持平衡则获得值为1的奖励，当杆子倒下则此episode结束。

In [1]:
import gym

env = gym.make('CartPole-v1')

print(env.action_space)
print(env.observation_space)

Discrete(2)
Box(4,)


Agent的策略用一个神经网络来表示：

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.autograd import Variable

# Hyperparameters
learning_rate = 0.01
gamma = 0.99

policy gradient：$\nabla_{\theta}J(\theta)=\frac{1}{N}\sum_{i=1}^N[(\sum_{t=1}^T \nabla_{\theta}\log\pi_{\theta}(a_{i,t}|s_{i,t}))(\sum_{t=1}^T r(s_{i,t}|a_{i,t}))]$

In [3]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.state_space = env.observation_space.shape[0]
        self.action_space = env.action_space.n
        
        self.l1 = nn.Linear(self.state_space, 128, bias=False)
        self.l2 = nn.Linear(128, self.action_space, bias=False)
        
        self.gamma = gamma
        
        # Episode policy and reward history 
        # 用来计算policy gradient
        self.log_probs = []
        self.reward_episode = []

    def forward(self, x):    
        model = torch.nn.Sequential(
            self.l1,
            nn.Dropout(p=0.6),
            nn.ReLU(),
            self.l2,
            nn.Softmax(dim=-1)
        )
        return model(x)

In [4]:
policy = Policy()
optimizer = optim.Adam(policy.parameters(), lr=learning_rate)

policy将为每个动作（向左或向右）返回一个概率，下面这个函数的输入为当前状态，将此状态输入policy得到输出action，再将其转化成一个动作返回。

In [5]:
from torch.distributions import Categorical

# Select an action (0 or 1) by running policy model and choosing based on the probabilities in state
def select_action(state):
    #Select an action (0 or 1) by running policy model and choosing based on the probabilities in state
    state = torch.from_numpy(state).type(torch.FloatTensor)
    probs = policy(state)
    m = Categorical(probs)         # 此处m是类别分布，假如probs=[0.3,0.7]，则对m采样可能获得0或1
    action = m.sample()            # P(action=0)=0.3 P(action=1)=0.7
    
    policy.log_probs.append(m.log_prob(action))    # 注意！！这里应该是对数似然！！
                                                   # action.item()返回0或1，m.log_prob(action)返回action所对应概率的对数
    return action.item()

Policy Gradient每次对policy的参数$\theta$做更新，更新公式如下：

$\Delta\theta = \alpha\nabla_{\theta}\log \pi_{\theta}(s_t,a_t)v_t$

$\log \pi_{\theta}(s_t,a_t)$在select_action函数中已经记在了policy_histroy内。这里主要关心$v_t$，其表达式如下：

$v_t=\sum_{k=0}^N \gamma^{k} r_t$

由此可见在第t步的reward为此步的reward和之后的reward的加权和。这样如果此episode越长，那么这一步得到的reward就越大。下面的代码用rewards来记录每一步的reward，rewards与policy_histroy的内积即为该网络的loss。由此可以得到policy gradient的一种直观理解——如果某一步的reward大，那么其action发生的概率应该尽可能大；反之则action发生概率应该尽可能小，这正是优化的目标！

loss为-log(prob)$*$reward，希望loss尽可能小就是希望log(prob)$*$reward尽可能大，即当reward为正时希望$\theta$使log(prob)尽量大；当reward为负时希望$\theta$使log(prob)尽量小！

In [6]:
import numpy as np

def update_policy():
    R = 0
    rewards = []
    
    # Discount future rewards back to the present using gamma
    for r in policy.reward_episode[::-1]:
        R = r + policy.gamma * R
        rewards.insert(0,R)
        
    # Scale rewards
    rewards = torch.FloatTensor(rewards)
    rewards = (rewards - rewards.mean()) / (rewards.std())
    
    # Calculate loss
    loss = []
    for log_prob, reward in zip(policy.log_probs, rewards):
        loss.append(-log_prob * reward)
        
    policy_loss = torch.stack(loss).sum()
    
    # Update network weights
    optimizer.zero_grad()
    policy_loss.backward()
    optimizer.step()
    
    del policy.reward_episode[:]
    del policy.log_probs[:]

下面的代码是训练网络的主循环，一共进行episodes轮，每轮1000步（杆子倒下则停止），每步记录下reward和policy；在此episode结束后更新policy。

In [7]:

def main(episodes):
    running_reward = 10
    for episode in range(episodes):
        state = env.reset() # Reset environment and record the starting state
        done = False       
    
        for time in range(1000):
            action = select_action(state)
            # Step through environment using chosen action
            state, reward, done, _ = env.step(action)

            # Save reward
            policy.reward_episode.append(reward)
            if done:
                break
        
        # Used to determine when the environment is solved.
        running_reward = (running_reward * 0.99) + (time * 0.01)

        update_policy()

        if episode % 50 == 0:
            print('Episode {}\tLast length: {:5d}\tAverage length: {:.2f}'.format(episode, time, running_reward))

        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and the last episode runs to {} time steps!".format(running_reward, time))
            break

开始运行模型：

In [8]:
episodes = 1000
main(episodes)

Episode 0	Last length:    27	Average length: 10.17
Episode 50	Last length:    38	Average length: 17.82
Episode 100	Last length:   100	Average length: 36.72
Episode 150	Last length:   248	Average length: 65.19
Episode 200	Last length:   499	Average length: 127.43
Episode 250	Last length:   147	Average length: 222.56
Episode 300	Last length:   499	Average length: 275.76
Episode 350	Last length:   499	Average length: 362.18
Episode 400	Last length:   499	Average length: 411.22
Episode 450	Last length:   499	Average length: 443.99
Episode 500	Last length:   499	Average length: 453.89
Episode 550	Last length:   499	Average length: 470.46
Solved! Running reward is now 475.183308304331 and the last episode runs to 499 time steps!
